In [1]:
!pip install k-means-constrained

In [ ]:
%pip install AMPL amplpy

from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"],
    license_uuid="") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

#Modelo AMPL

In [3]:
ampl_code_estendido = """# ==============================================================================
# MODELO AMPL: ALOCAÇÃO DE LOJAS PARA PROMOTORES DE VENDAS
# ==============================================================================

# --- 1. CONJUNTOS ---
set PROMOTORES;             # Conjunto de promotores disponíveis
set LOJAS;                  # Conjunto de lojas a visitar
set DIAS := 1..6;           # Dias da semana (1=Seg, 6=Sáb)
set FREQ_POSSIVEIS := 1..6; # Opções de frequência de visita (menu de escolha)
set LOJAS_FIXAS within LOJAS default {}; # Conjunto usado pela Heurística LNS para travar parte da solução

# --- 2. PARÂMETROS GERAIS ---
param M; # Número total de lojas
param N; # Número máximo de promotores
param promotor_fixo{LOJAS_FIXAS} integer; # Parâmetro usado pelo LNS para forçar lojas a promotores específicos

# --- 3. DADOS DAS LOJAS E GEOGRAFIA ---
param tipo{LOJAS} symbolic; # P, M ou G
param X{LOJAS};             # Latitude/X
param Y{LOJAS};             # Longitude/Y
param tempo_visita{LOJAS};  # Tempo de atendimento (minutos)

# MATRIZ DE LUCRO
param lucro_potencial{LOJAS, FREQ_POSSIVEIS};

# Frequência mínima de visitas por loja
param freq_minima{LOJAS};

# Distância e Tempo de Deslocamento
param D{j in LOJAS, k in LOJAS} := sqrt((X[j] - X[k])^2 + (Y[j] - Y[k])^2);
param tau := 0.084694 * M + 9.938776; # Tempo médio de deslocamento entre lojas

# Jornada de trabalho (minutos)
param H{d in DIAS} := if d <= 5 then 480 else 240;

# Limite físico de lojas na carteira
param max_lojas := 8;

# --- 4. PESOS DA FUNÇÃO OBJETIVO ---
param P_rep  := 750;   # Custo fixo de contratar um promotor
param P_dist := 0.06;  # Custo por unidade de distância
param P_he   := 20.45; # Custo por minuto de hora extra
param PB     := 5.0;   # Penalidade por desbalanceamento (R$/min de desvio)

# --- 5. VARIÁVEIS DE DECISÃO ---

# Variáveis Operacionais
var r{PROMOTORES} binary;              # 1 se o promotor for contratado
var c{PROMOTORES, LOJAS} binary;       # 1 se o promotor atende a loja
var v{PROMOTORES, LOJAS, DIAS} binary; # 1 se visita a loja no dia D
var p{PROMOTORES, LOJAS, LOJAS} binary;# Linearização quadrática de distância
var h{PROMOTORES, DIAS} >= 0;          # Horas extras (minutos)

# Variável Estratégica (Nível de Serviço)
# z[j, f] = 1 se escolhemos visitar a loja j exatamente f vezes
var z{LOJAS, FREQ_POSSIVEIS} binary;

# Variáveis Auxiliares de Balanceamento (Linearização Min-Max)
var CargaTotal{PROMOTORES} >= 0;
var CargaMax >= 0;
var CargaMin >= 0;

# --- 6. FUNÇÃO OBJETIVO ---
# Maximizar Lucro: Receita Obtida - (Custos Operacionais + Penalidades)

maximize lucro_total:
    # RECEITA
    ( sum{j in LOJAS, f in FREQ_POSSIVEIS} lucro_potencial[j, f] * z[j, f] ) -
    # CUSTOS
    ( P_rep * sum{i in PROMOTORES} r[i] ) -
    ( P_dist * sum{i in PROMOTORES, j in LOJAS, k in LOJAS} p[i,j,k] * D[j,k] ) -
    ( P_he * sum{i in PROMOTORES, d in DIAS} h[i,d] ) -
    ( PB * (CargaMax - CargaMin) ); # Penalidade pela injustiça na carga

# --- 7. RESTRIÇÕES ---

# R1: Cada loja deve ser atribuída a exatamente um promotor
subject to alocacao_unica{j in LOJAS}:
     sum{i in PROMOTORES} c[i,j] = 1;

# R2: Jornada de Trabalho e Horas Extras
subject to Jornada_Diaria{i in PROMOTORES, d in DIAS}:
    sum{j in LOJAS} v[i,j,d] * (tempo_visita[j] + tau) <= H[d] * r[i] + h[i,d];

# R3: Limite de Carteira
subject to Limite_Lojas{i in PROMOTORES}:
    sum{j in LOJAS} c[i,j] <= max_lojas * r[i];

# --- NOVAS RESTRIÇÕES DE FREQUÊNCIA (NÍVEL DE SERVIÇO) ---

# R4a: Deve-se escolher exatamente UMA frequência para cada loja
subject to Escolha_Unica_Freq{j in LOJAS}:
    sum{f in FREQ_POSSIVEIS} z[j, f] = 1;

# R4a_min: Deve-se respeitar a frequência mínima
subject to Freq_Minima_Respeitada{j in LOJAS, f in FREQ_POSSIVEIS: f < freq_minima[j]}:
    z[j, f] = 0;

# R4b: Consistência Global (Total de visitas realizadas = Frequência escolhida)
subject to Consistencia_Visitas_Total{j in LOJAS}:
    sum{i in PROMOTORES, d in DIAS} v[i,j,d] = sum{f in FREQ_POSSIVEIS} f * z[j,f];

# R4c: Consistência Local (Visita válida apenas se for o responsável pela loja)
subject to Visita_Se_Dono{i in PROMOTORES, j in LOJAS, d in DIAS}:
    v[i,j,d] <= c[i,j];

# --- RESTRIÇÕES DE DISTÂNCIA (LINEARIZAÇÃO) ---

subject to Ativa_P_J{i in PROMOTORES, j in LOJAS, k in LOJAS}:
    p[i,j,k] <= c[i,j];

subject to Ativa_P_K{i in PROMOTORES, j in LOJAS, k in LOJAS}:
    p[i,j,k] <= c[i,k];

subject to Ativa_P_Ambos{i in PROMOTORES, j in LOJAS, k in LOJAS}:
    p[i,j,k] >= c[i,j] + c[i,k] - 1;

# --- RESTRIÇÕES DE BALANCEAMENTO (CARGA) ---

# R9: Calcular a carga total de cada promotor (Visita + Deslocamento)
subject to Def_CargaTotal{i in PROMOTORES}:
    CargaTotal[i] = sum{j in LOJAS, d in DIAS} v[i,j,d] * (tempo_visita[j] + tau);

# R10: Definir Teto (CargaMax >= Carga de qualquer contratado)
subject to Def_CargaMax{i in PROMOTORES}:
    CargaMax >= CargaTotal[i] - (1 - r[i]) * 10000;

# R11: Definir Piso (CargaMin <= Carga de qualquer contratado)
subject to Def_CargaMin{i in PROMOTORES}:
    CargaMin <= CargaTotal[i] + (1 - r[i]) * 10000;

# --- RESTRIÇÕES DE HEURÍSTICA (LNS) ---

# R12: Trava as lojas fixas aos seus promotores
subject to FixarLojas_LNS {j in LOJAS_FIXAS}:
    c[promotor_fixo[j], j] = 1;
"""

# Salvar o código AMPL em um arquivo
with open('modelo_promotores_estendido.mod', 'w') as f:
     f.write(ampl_code_estendido)

print("Modelo salvo: modelo_promotores_estendido.mod")

Modelo salvo: modelo_promotores_estendido.mod


#Dados

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Carregar base de dados
df_completo = pd.read_csv('')

print(f"CSV carregado: {len(df_completo)} lojas")

CSV carregado: 20 lojas


#K-Means

In [6]:
from k_means_constrained import KMeansConstrained
import numpy as np

max_lojas_por_cluster = 15
min_lojas_por_cluster = 10
num_lojas = len(df_completo)

# Calcular número de clusters necessários
num_clusters = int(np.ceil(num_lojas / ((max_lojas_por_cluster + min_lojas_por_cluster)/2)))


print(f"Tamanho por cluster: {min_lojas_por_cluster}–{max_lojas_por_cluster}")

# Dados para clusterização
X = df_completo[['x_coordinate', 'y_coordinate']].values

# Aplicar K-Means com restrições
kmeans = KMeansConstrained(
    n_clusters=num_clusters,
    size_min=min_lojas_por_cluster,
    size_max=max_lojas_por_cluster,
    random_state=42
)
df_completo['cluster'] = kmeans.fit_predict(X)

# Estatísticas
distribuicao = df_completo['cluster'].value_counts().sort_index()

print(f"\nCLUSTERiZAÇÃO EXECUTADA\n");
print(f"\nMenor cluster: {distribuicao.min()} lojas")
print(f"Maior cluster: {distribuicao.max()} lojas")
print(f"Média: {distribuicao.mean():.1f} lojas")

Tamanho por cluster: 10–15

CLUSTERiZAÇÃO EXECUTADA


Menor cluster: 10 lojas
Maior cluster: 10 lojas
Média: 10.0 lojas


In [7]:
# Função para criar arquivo .dat para cada cluster

def criar_arquivo_dat_cluster(df_cluster, cluster_id):
    """
    Cria arquivo .dat para um cluster específico

    Parâmetros:
    - df_cluster: DataFrame com as lojas do cluster
    - cluster_id: ID do cluster
    """
    M = len(df_cluster)
    N = max(int(M / 4), 10)  # Número máximo de promotores

    df_cluster = df_cluster.copy()

    mapa_colunas = {'visit_duration_minutes': 'tempo_visita',
                    'initial_frequency': 'freq_visita'}

    df_cluster.rename(columns=mapa_colunas, inplace = True)

    # Validação de segurança
    if 'tempo_visita' not in df_cluster.columns or 'freq_visita' not in df_cluster.columns:
        raise ValueError(f"ERRO: As colunas de tempo ou frequência não foram encontradas no Cluster {cluster_id}")

    # Nome do arquivo
    nome_arquivo = f'dados_cluster_{cluster_id}.dat'

    # Criar arquivo .dat
    with open(nome_arquivo, 'w') as f:
        f.write(f"# Arquivo de dados - Cluster {cluster_id} ({M} lojas)\n")
        f.write(f"# Gerado automaticamente via K-Means\n\n")

        # Definir conjuntos
        f.write(f"set PROMOTORES := ")
        f.write(" ".join(str(i) for i in range(1, N+1)))
        f.write(";\n\n")

        f.write(f"set LOJAS := ")
        f.write(" ".join(str(i) for i in range(1, M+1)))
        f.write(";\n\n")

        # Parâmetros básicos
        f.write(f"param M := {M};\n")
        f.write(f"param N := {N};\n\n")

        # Coordenadas X
        f.write("param X :=\n")
        for idx, (_, row) in enumerate(df_cluster.iterrows(), 1):
            f.write(f"  {idx}  {row['x_coordinate']:.2f}\n")
        f.write(";\n\n")

        # Coordenadas Y
        f.write("param Y :=\n")
        for idx, (_, row) in enumerate(df_cluster.iterrows(), 1):
            f.write(f"  {idx}  {row['y_coordinate']:.2f}\n")
        f.write(";\n\n")

        # Tempo de visita
        f.write("param tempo_visita :=\n")
        for idx, (_, row) in enumerate(df_cluster.iterrows(), 1):
            f.write(f"  {idx}  {row['tempo_visita']}\n")
        f.write(";\n\n")

        # Matriz de lucro potencial
        lucro_cols = [f'profitability_freq_{f}' for f in range(1, 7)]

        f.write("# Lucro potencial (R$) para cada loja em funcao da frequencia (1 a 6 visitas)\n")
        f.write("param lucro_potencial: ")
        f.write(" ".join(str(f) for f in range(1, 7)))  # Cabeçalho 1 2 3 4 5 6
        f.write(" :=\n")
        for idx, (_, row) in enumerate(df_cluster.iterrows(), 1):
            line = f"  {idx} "
            line += " ".join(f"{row[col]:.2f}" for col in lucro_cols)
            f.write(line + "\n")
        f.write(";\n\n")

        # Frequência mínima
        f.write("param freq_minima :=\n")
        for idx, (_, row) in enumerate(df_cluster.iterrows(), 1):
            f.write(f"  {idx}  {int(row['freq_visita'])}\n")
        f.write(";\n\n")

        # Tipo da loja
        f.write("param tipo :=\n")
        for idx, (_, row) in enumerate(df_cluster.iterrows(), 1):
            f.write(f"  {idx}  \"{row['type']}\"\n")
        f.write(";\n\n")

    return nome_arquivo, M, N

# Criar arquivos .dat para cada cluster
arquivos_clusters = []

for cluster_id in sorted(df_completo['cluster'].unique()):
    df_cluster = df_completo[df_completo['cluster'] == cluster_id].reset_index(drop=True)
    nome_arquivo, M, N = criar_arquivo_dat_cluster(df_cluster, cluster_id)

    arquivos_clusters.append({
        'cluster_id': cluster_id,
        'arquivo': nome_arquivo,
        'num_lojas': M,
        'num_promotores_max': N,
        'distribuicao': df_cluster['type'].value_counts().to_dict()
    })

print(f"\n Total de arquivos criados: {len(arquivos_clusters)}")


 Total de arquivos criados: 2


#LNS + SA

##Solução S0

In [8]:
import time
import pandas as pd


df_completo['id_promotor_global'] = -1  # Coluna para armazenar o promotor
df_completo['id_cluster_inicial'] = -1 # Coluna para rastrear o cluster original

# Contador global para ID único de promotor
promotor_id_global_contador = 0

# Lista para guardar os resultados
resultados_clusters = []

# Resolver CLUSTERS
tempo_total_inicio = time.time()

for idx, cluster_info in enumerate(arquivos_clusters):
    cluster_id = cluster_info['cluster_id']
    tempo_inicio = time.time()

    try:
        # Preparar comandos AMPL
        ampl_commands = f"""
reset;
model modelo_promotores_estendido.mod;
data {cluster_info['arquivo']};
option solver gurobi;
option gurobi_options 'mipgap=0.01 timelimit= 60';
solve;
"""

        # Executar modelo
        ampl.eval(ampl_commands)
        tempo_fim = time.time()
        tempo_execucao = tempo_fim - tempo_inicio

        # Extrair resultados escalares
        num_promotores = ampl.get_value("sum{i in PROMOTORES} r[i]")
        lucro_total = ampl.get_objective("lucro_total").value()

        # Obter DataFrame das lojas do cluster
        lojas_deste_cluster_df = df_completo[df_completo['cluster'] == cluster_id].reset_index()

        # Criar mapa ID global
        mapa_local_para_global = dict(zip(range(1, len(lojas_deste_cluster_df) + 1), lojas_deste_cluster_df['index']))

        # Obter as variáveis de decisão do AMPL
        var_r = ampl.get_variable("r").get_values().to_pandas()
        var_c = ampl.get_variable("c").get_values().to_pandas()

        # Detectar nome da coluna
        col_r_val = [c for c in var_r.columns if 'val' in c.lower()][0]
        col_c_val = [c for c in var_c.columns if 'val' in c.lower()][0]

        # Filtrar promotores usados e suas atribuições
        var_r_usados = var_r[var_r[col_r_val] > 0.9]
        promotores_usados_local = var_r_usados.index.get_level_values(0).unique()
        atribuicoes = var_c[var_c[col_c_val] > 0.9]

        # Mapear de volta para o DataFrame global
        for p_local in promotores_usados_local:
            # Incrementar o contador para criar um ID global único
            promotor_id_global_contador += 1

            # Encontrar todas as lojas locais (j) atribuídas a este promotor (p_local)
            try:
                # Tenta acessar as atribuições deste promotor
                lojas_do_promotor_local = atribuicoes.loc[p_local].index

                # Garantir que 'lojas_do_promotor_local' seja sempre iterável
                if isinstance(lojas_do_promotor_local, (int, str, float)):
                    lojas_do_promotor_local = [lojas_do_promotor_local] # Caso de 1 loja

            except KeyError:
                continue

            for j_local in lojas_do_promotor_local:
                # Obter o ID global da loja
                id_global_loja = mapa_local_para_global[j_local]

                # Armazenar a atribuição na Solução S_0
                df_completo.at[id_global_loja, 'id_promotor_global'] = promotor_id_global_contador
                df_completo.at[id_global_loja, 'id_cluster_inicial'] = cluster_id


        # Armazenar resultados
        resultado = {
            'cluster_id': cluster_id,
            'num_lojas': cluster_info['num_lojas'],
            'num_promotores': num_promotores,
            'lucro_total': lucro_total,
            'tempo_execucao': tempo_execucao,
            'distribuicao_lojas': cluster_info['distribuicao'],
            'status': 'Sucesso'
        }

        resultados_clusters.append(resultado)

        print(f"\nCLUSTER {cluster_id}")
        print(f"   Promotores necessários: {num_promotores}")
        print(f"   lucro total: R$ {lucro_total:.2f}")
        print(f"   Tempo de execução: {tempo_execucao:.2f}s")

    except Exception as e:
        print(f"\nERRO ao resolver CLUSTER {cluster_id}: {str(e)}")
        resultado = {
            'cluster_id': cluster_id,
            'num_lojas': cluster_info['num_lojas'],
            'num_promotores': None,
            'lucro_total': None,
            'tempo_execucao': time.time() - tempo_inicio,
            'distribuicao_lojas': cluster_info['distribuicao'],
            'status': f'Erro: {str(e)}'
        }
        resultados_clusters.append(resultado)

tempo_total_fim = time.time()
tempo_total_execucao = tempo_total_fim - tempo_total_inicio

print(f"Total de clusters: {len(resultados_clusters)}")
print(f"Tempo total de execução: {tempo_total_execucao:.2f}s ({tempo_total_execucao/60:.2f} min)")

#Calcular lucro total e criar a Solução S_0
lucro_total_S0 = sum(r['lucro_total'] for r in resultados_clusters if r['status'] == 'Sucesso')
df_solucao_best = df_completo.copy()

print(f"lucro total S_0: R$ {lucro_total_S0:.2f}")
print(f"Total de promotores: {df_solucao_best['id_promotor_global'].nunique()}")
print("DataFrame 'df_solucao_best' criado.")

Gurobi 13.0.0:   mip:gap = 0.01
  lim:time = 60
Gurobi 13.0.0: optimal solution; objective 1821.083578
40051 simplex iterations
1343 branching nodes
absmipgap=18.0765, relmipgap=0.00992625

CLUSTER 0
   Promotores necessários: 2
   lucro total: R$ 1821.08
   Tempo de execução: 13.49s
Gurobi 13.0.0:   mip:gap = 0.01
  lim:time = 60
Gurobi 13.0.0: optimal solution; objective 4462.899005
45097 simplex iterations
1303 branching nodes
absmipgap=43.9653, relmipgap=0.00985129

CLUSTER 1
   Promotores necessários: 3
   lucro total: R$ 4462.90
   Tempo de execução: 13.68s
Total de clusters: 2
Tempo total de execução: 27.22s (0.45 min)
lucro total S_0: R$ 6283.98
Total de promotores: 5
DataFrame 'df_solucao_best' criado.


##Função Destroy

In [9]:
import numpy as np
from scipy.spatial import distance

def destroy_geografico(df_solucao_atual, num_lojas_para_liberar):
    """
    Seleciona um conjunto de lojas para "liberar" (destruir)
    baseado em proximidade geográfica.

    1. Escolhe 1 loja aleatória como "pivô".
    2. Encontra as (N-1) lojas mais próximas dela.
    3. Retorna os IDs (índices) dessas N lojas.

    Parâmetros:
    - df_solucao_atual: DataFrame contendo a solução (ex: df_solucao_best)
    - num_lojas_para_liberar: Quantidade de lojas a serem liberadas (ex: 20)

    Retorna:
    - Um objeto de índice (Index) com os IDs das lojas liberadas.
    """

    # Escolher uma loja "pivô" aleatória
    loja_pivo = df_solucao_atual.sample(1)
    loja_pivo_index = loja_pivo.index[0]

    # Obter as coordenadas (x, y) do pivô
    pivo_coords = loja_pivo[['x_coordinate', 'y_coordinate']].values

    # Calcular a distância de todas as lojas para o pivô
    all_coords = df_solucao_atual[['x_coordinate', 'y_coordinate']].values

    # cdist calcula a matriz de distâncias.
    distancias = distance.cdist(pivo_coords, all_coords, 'euclidean')[0]

    # Pegar os índices das N lojas mais próximas.
    indices_proximos = np.argsort(distancias)

    # Pegar os primeiros N índices
    indices_das_lojas_liberadas = indices_proximos[:num_lojas_para_liberar]

    # Retornar os IDs reais  dessas lojas
    lojas_liberadas_ids = df_solucao_atual.iloc[indices_das_lojas_liberadas].index

    print(f"Destroy: Pivô ID {loja_pivo_index}. Liberando {len(lojas_liberadas_ids)} lojas.")

    return lojas_liberadas_ids

##Função Repair

In [10]:
import numpy as np
import pandas as pd
import time
from typing import Tuple, List

def repair_com_ampl(df_solucao_atual: pd.DataFrame,
                    lojas_liberadas_ids: List[int],
                    ampl_instance,
                    df_base_completo: pd.DataFrame,
                    timelimit_reparo: int = 60) -> Tuple[pd.DataFrame, float]:
    """
    Executa a etapa 'Repair' do LNS.
    """
    print(f"  [Repair] Iniciando reparo para {len(lojas_liberadas_ids)} lojas...")

    # Definir lojas fixas
    lojas_liberadas_ids = list(lojas_liberadas_ids)
    df_solucao_atual = df_solucao_atual.copy()

    try:
        df_fixas = df_solucao_atual.drop(index=lojas_liberadas_ids)
    except Exception:
        df_fixas = df_solucao_atual.loc[~df_solucao_atual.index.isin(lojas_liberadas_ids)]

    lojas_fixas_ids_globais = list(df_fixas.index)

    # Mapeamento global para local
    mapa_global_para_local = {idx_global: local_id for local_id, idx_global
                              in enumerate(df_base_completo.index, start=1)}
    mapa_local_para_global_reverso = {local: glob for glob, local in mapa_global_para_local.items()}

    M = len(df_base_completo)
    N = int(df_solucao_atual['id_promotor_global'].max())
    if N <= 0 or np.isnan(N):
        N = max(int(M / 4), 10)

    nome_arquivo_dat_LNS = "dados_LNS_repair_temp.dat"
    df_base = df_base_completo.copy()

    # Renomear para o padrão esperado pelo script de escrita
    mapa_colunas = {
        'visit_duration_minutes': 'tempo_visita',
        'initial_frequency': 'freq_visita'
    }
    df_base.rename(columns=mapa_colunas, inplace=True)

    # Validação básica
    if 'tempo_visita' not in df_base.columns or 'freq_visita' not in df_base.columns:
         raise ValueError("df_base_completo precisa conter colunas de tempo e frequência (verifique os nomes).")

    # Garantir tipos numéricos
    df_base['tempo_visita'] = df_base['tempo_visita'].fillna(0).astype(int)
    df_base['freq_visita'] = df_base['freq_visita'].fillna(0).astype(int)

    #Gerar .dat para o AMPL
    try:
        with open(nome_arquivo_dat_LNS, 'w', encoding='utf-8') as f:
            f.write(f"# Arquivo de dados - LNS Repair (M={M} lojas, N={N} promotores)\n\n")

            f.write("set PROMOTORES := ")
            f.write(" ".join(str(i) for i in range(1, N + 1)))
            f.write(";\n\n")

            f.write("set LOJAS := ")
            f.write(" ".join(str(i) for i in range(1, M + 1)))
            f.write(";\n\n")

            f.write(f"param M := {M};\n")
            f.write(f"param N := {N};\n\n")

            # Coordenadas X
            f.write("param X :=\n")
            for idx_global, row in df_base.iterrows():
                idx_local = mapa_global_para_local[idx_global]
                f.write(f"  {idx_local}  {float(row['x_coordinate']):.6f}\n")
            f.write(";\n\n")

            # Coordenadas Y
            f.write("param Y :=\n")
            for idx_global, row in df_base.iterrows():
                idx_local = mapa_global_para_local[idx_global]
                f.write(f"  {idx_local}  {float(row['y_coordinate']):.6f}\n")
            f.write(";\n\n")

            # tempo_visita
            f.write("param tempo_visita :=\n")
            for idx_global, row in df_base.iterrows():
                idx_local = mapa_global_para_local[idx_global]
                f.write(f"  {idx_local}  {int(row['tempo_visita'])}\n")
            f.write(";\n\n")

            # Matriz de lucro potencial
            lucro_cols = [f'profitability_freq_{f}' for f in range(1, 7)]

            f.write("# Lucro potencial (R$) para cada loja em funcao da frequencia (1 a 6 visitas)\n")
            f.write("param lucro_potencial: ")
            f.write(" ".join(str(f) for f in range(1, 7)))
            f.write(" :=\n")
            for idx_global, row in df_base.iterrows():
                idx_local = mapa_global_para_local[idx_global]
                line = f"  {idx_local} "
                line += " ".join(f"{row[col]:.2f}" for col in lucro_cols)
                f.write(line + "\n")
            f.write(";\n\n")

            # Frequência mínima
            f.write("param freq_minima :=\n")
            for idx_global, row in df_base.iterrows():
                idx_local = mapa_global_para_local[idx_global]
                f.write(f"  {idx_local}  {int(row['freq_visita'])}\n")
            f.write(";\n\n")

            # DADOS DE FIXAÇÃO LNS
            f.write("\n# --- DADOS DE FIXAÇÃO LNS ---\n")

            if len(lojas_fixas_ids_globais) > 0:
                ids_fixas_locais = [mapa_global_para_local[idx] for idx in lojas_fixas_ids_globais]
                f.write("set LOJAS_FIXAS := ")
                f.write(" ".join(map(str, ids_fixas_locais)))
                f.write(";\n\n")
            else:
                f.write("set LOJAS_FIXAS := ;\n\n")

            f.write("param promotor_fixo :=\n")
            for idx_global in lojas_fixas_ids_globais:
                idx_local = mapa_global_para_local[idx_global]
                id_promotor = df_solucao_atual.at[idx_global, 'id_promotor_global']

                # Tratamento de NaN/Float para Int
                try:
                    if pd.isna(id_promotor) or id_promotor < 0:
                        id_promotor_int = 0
                    else:
                        id_promotor_int = int(id_promotor)
                except Exception:
                    id_promotor_int = 0

                f.write(f"  {idx_local}  {id_promotor_int}\n")
            f.write(";\n\n")

    except Exception as e:
        print(f"  [Repair] ERRO ao GERAR .dat LNS: {e}")
        return None, float('inf')

    # Chamar o AMPL
    try:
        ampl_commands = f"""
        reset;
        model modelo_promotores_estendido.mod;
        data {nome_arquivo_dat_LNS};

        option solver gurobi;
        option gurobi_options 'mipgap=0.01';
        solve;
        """
        ampl_instance.eval(ampl_commands)

        # Extrair lucro
        try:
            novo_lucro_total = float(ampl_instance.get_objective("lucro_total").value())
        except Exception:
            novo_lucro_total = float('inf')

        # Construir DataFrame de saída
        df_solucao_nova = df_base.copy()
        df_solucao_nova['id_promotor_global'] = -1

        # Ler variáveis do AMPL
        var_r_df = ampl_instance.get_variable("r").get_values().to_pandas()
        var_c_df = ampl_instance.get_variable("c").get_values().to_pandas()

        # Função auxiliar para detectar coluna de valor
        def detectar_col_val(df: pd.DataFrame) -> str:
            for c in df.columns:
                if 'val' in c.lower(): return c
            return df.columns[0]

        col_r_val = detectar_col_val(var_r_df)
        col_c_val = detectar_col_val(var_c_df)

        # Identificar promotores usados
        try:
            s_r = var_r_df[col_r_val]
            promotores_usados_local = s_r[s_r > 0.9].index.tolist()
        except Exception:
            promotores_usados_local = []

        # Mapear atribuições
        c_mask = var_c_df[col_c_val] > 0.9
        atribuicoes_ativas = var_c_df[c_mask]

        for idx, _ in atribuicoes_ativas.iterrows():
            try:
                if isinstance(idx, tuple):
                    p_local, j_local = idx
                else:
                    p_local, j_local = idx

                id_global_loja = mapa_local_para_global_reverso.get(int(j_local))

                if id_global_loja is not None:
                    df_solucao_nova.at[id_global_loja, 'id_promotor_global'] = int(p_local)
            except Exception:
                continue

        print(f"Reparo concluído. Novo lucro: {novo_lucro_total if np.isfinite(novo_lucro_total) else 'N/A'}")
        return df_solucao_nova, novo_lucro_total

    except Exception as e:
        print(f"ERRO ao RESOLVER no AMPL: {e}")
        return None, float('inf')

##Resultado

In [11]:
import math
import random
import time


# Parâmetros do LNS
N_ITERACOES_LNS = len(df_completo)     # Número máximo de iterações
N_LOJAS_PARA_LIBERAR = 10
TIMELIMIT_REPARO = 60

# Critério de Parada por Convergência
num_lojas = len(df_completo)
MAX_ITERACOES_SEM_MELHORA = max(10, int(num_lojas * 0.25))  # Mínimo 10, ou 25% do número de lojas
iteracoes_sem_melhora = 0

# Limite de Tempo Total do LNS
TEMPO_MAXIMO_LNS_SEGUNDOS = 20 * 60  # 20 minutos em segundos
tempo_inicio_lns = time.time()

# Tolerância para comparação de ponto flutuante
TOLERANCIA = 0.01  # diferenças menores são consideradas iguais

# Parâmetros do Simulated Annealing (SA)
lucro_inicial_S0 = lucro_total_S0

# Temperatura inicial (10% do lucro inicial)
T_inicial = lucro_inicial_S0 * 0.10
T_final = 0.1

# Taxa de resfriamento para ir de T_inicial a T_final em N iterações
taxa_resfriamento = (T_final / T_inicial)**(1.0 / N_ITERACOES_LNS)

# Inicialização das Soluções

# df_best / lucro_best: Guarda a melhor solução já encontrada
lucro_best = lucro_inicial_S0
df_best = df_solucao_best.copy()

# df_atual / lucro_atual: Guarda a solução da iteração atual
lucro_atual = lucro_inicial_S0
df_atual = df_solucao_best.copy()

T_atual = T_inicial

# DataFrame base
df_completo_base = df_completo.drop(columns=['id_promotor_global', 'id_cluster_inicial', 'cluster'])

print(f"Iterações máximas: {N_ITERACOES_LNS}")
print(f"Lojas por 'destroy': {N_LOJAS_PARA_LIBERAR}")
print(f"Lucro inicial S_0: R$ {lucro_inicial_S0:.2f}")
print(f"Temp. Inicial (T0): {T_inicial:.2f}")
print(f"Temp. Final (Tf): {T_final:.2f}")
print(f"Taxa de Resfriamento: {taxa_resfriamento:.6f}")
print(f"Tolerância de comparação: R$ {TOLERANCIA:.2f}")
print(f"Critério de parada: {MAX_ITERACOES_SEM_MELHORA} iterações sem melhora")
print(f"Tempo máximo: {TEMPO_MAXIMO_LNS_SEGUNDOS/60:.0f} minutos")

# LOOP PRINCIPAL
for i in range(N_ITERACOES_LNS):

    # Verificar limite de tempo
    tempo_decorrido = time.time() - tempo_inicio_lns
    if tempo_decorrido > TEMPO_MAXIMO_LNS_SEGUNDOS:
        print(f"TEMPO LIMITE: {TEMPO_MAXIMO_LNS_SEGUNDOS/60:.0f} minutos atingidos.")
        print(f" Parando na iteração {i+1}.")
        break

    print(f"\n--- Iteração SA {i+1}/{N_ITERACOES_LNS} (Temp: {T_atual:.2f}) (Tempo: {tempo_decorrido/60:.1f} min) ---")

    # 1. DESTROY
    lojas_para_liberar = destroy_geografico(df_atual, N_LOJAS_PARA_LIBERAR)

    # 2. REPAIR
    df_nova, lucro_novo = repair_com_ampl(
        df_atual,
        lojas_para_liberar,
        ampl,
        df_completo_base,
        TIMELIMIT_REPARO
    )

    # Verificar se o reparo falhou
    if not np.isfinite(lucro_novo):
        print("  [Repair] Falha no reparo, pulando iteração.")
        iteracoes_sem_melhora += 1

        # Verificar convergência mesmo em caso de falha
        if iteracoes_sem_melhora >= MAX_ITERACOES_SEM_MELHORA:
            print(f"CONVERGÊNCIA: {MAX_ITERACOES_SEM_MELHORA} iterações sem melhora.")
            print(f"   Parando antecipadamente na iteração {i+1}.")
            break
        continue

    # 3. DECIDIR
    delta_lucro = lucro_novo - lucro_atual

    # CASO 1: Solução MELHOR
    if delta_lucro > TOLERANCIA:
        lucro_atual = lucro_novo
        df_atual = df_nova.copy()

        # Verifica se é a Melhor Global
        if lucro_novo > lucro_best + TOLERANCIA:
            df_best = df_nova.copy()
            lucro_best = lucro_novo
            iteracoes_sem_melhora = 0
            print(f"NOVA MELHOR SOLUÇÃO GLOBAL (Lucro: R$ {lucro_best:.2f}).")
        else:
            iteracoes_sem_melhora += 1
            print(f"  [Accept] Solução melhor que a anterior (Lucro: R$ {lucro_atual:.2f}).")
            print(f"  Melhor lucro global: R$ {lucro_best:.2f} (sem melhora há {iteracoes_sem_melhora} iterações).")

    # CASO 2: Solução EQUIVALENTE
    elif delta_lucro >= -TOLERANCIA:
        lucro_atual = lucro_novo
        df_atual = df_nova.copy()
        iteracoes_sem_melhora += 1  # Conta como sem melhora para convergência
        print(f"  [Accept] Solução equivalente (delta_lucro: R$ {delta_lucro:.6f}).")
        print(f"  Melhor lucro global: R$ {lucro_best:.2f} (sem melhora há {iteracoes_sem_melhora} iterações).")

    # CASO 3: Solução PIOR
    else:
        iteracoes_sem_melhora += 1  # Conta como sem melhora para convergência

        # Calcular probabilidade de aceitar solução pior
        prob_aceite = math.exp(delta_lucro / T_atual)

        if random.random() < prob_aceite:
            lucro_atual = lucro_novo
            df_atual = df_nova.copy()
            print(f"  [Accept] Aceitando solução pior (lucro: R$ {lucro_novo:.2f}) Prob: {prob_aceite:.2%}.")
        else:
            print(f"  [Reject] Solução pior foi rejeitada.")

        print(f"  Melhor lucro global: R$ {lucro_best:.2f} (sem melhora há {iteracoes_sem_melhora} iterações).")

    # Verificar critério de convergência
    if iteracoes_sem_melhora >= MAX_ITERACOES_SEM_MELHORA:
        print(f"CONVERGÊNCIA: {MAX_ITERACOES_SEM_MELHORA} iterações sem melhora.")
        print(f" Parando antecipadamente na iteração {i+1}.")
        break

    # Resfriar a temperatura
    T_atual = T_atual * taxa_resfriamento

    # Garantir que a temperatura não caia abaixo do final
    if T_atual < T_final:
        T_atual = T_final

# Resumo Final
tempo_total = time.time() - tempo_inicio_lns
print(f"Tempo total de execução: {tempo_total/60:.2f} minutos")
print(f"Iterações executadas: {i+1}")
print(f"Lucro Inicial (S_0): R$ {lucro_inicial_S0:.2f}")
print(f"Lucro Final (S_best): R$ {lucro_best:.2f}")
print(f"Melhoria: R$ {lucro_best - lucro_inicial_S0:.2f}")

# A sua solução final e otimizada está no DataFrame 'df_best'
# O lucro final está na variável 'lucro_best'

Iterações máximas: 20
Lojas por 'destroy': 10
Lucro inicial S_0: R$ 6283.98
Temp. Inicial (T0): 628.40
Temp. Final (Tf): 0.10
Taxa de Resfriamento: 0.645785
Tolerância de comparação: R$ 0.01
Critério de parada: 10 iterações sem melhora
Tempo máximo: 20 minutos

--- Iteração SA 1/20 (Temp: 628.40) (Tempo: 0.0 min) ---
Destroy: Pivô ID 11. Liberando 10 lojas.
  [Repair] Iniciando reparo para 10 lojas...
Gurobi 13.0.0:   mip:gap = 0.01
Gurobi 13.0.0: optimal solution; objective 6286.606255
209173 simplex iterations
4519 branching nodes
absmipgap=62.8542, relmipgap=0.00999812
Reparo concluído. Novo lucro: 6286.606255374434
NOVA MELHOR SOLUÇÃO GLOBAL (Lucro: R$ 6286.61).

--- Iteração SA 2/20 (Temp: 405.81) (Tempo: 0.3 min) ---
Destroy: Pivô ID 8. Liberando 10 lojas.
  [Repair] Iniciando reparo para 10 lojas...
Gurobi 13.0.0:   mip:gap = 0.01
Gurobi 13.0.0: optimal solution; objective 6299.326234
534801 simplex iterations
15150 branching nodes
absmipgap=62.9857, relmipgap=0.0099988
Reparo c

In [12]:
# Renumeração Sequencial de promotores

if 'df_best' in locals():

    # Identificar quais IDs de promotores estão sendo usados
    promotores_ativos = df_best[df_best['id_promotor_global'] > 0]['id_promotor_global'].unique()

    #Ordenar esses IDs
    promotores_ativos_sorted = sorted(promotores_ativos)

    #Criar um mapa
    mapa_renumeracao = {antigo: novo for novo, antigo in enumerate(promotores_ativos_sorted, start=1)}

    # Aplicar a renumeração no DataFrame
    def aplicar_mapa(id_antigo):
        if id_antigo in mapa_renumeracao:
            return mapa_renumeracao[id_antigo]
        return -1

    df_best['id_promotor_global'] = df_best['id_promotor_global'].apply(aplicar_mapa)

    # Atualizar o número total de promotores usados
    num_final_promotores = df_best['id_promotor_global'].max()

    print(f"Total de contratados: {num_final_promotores}")

else:
    print("Variável 'df_best' não encontrada.")

Realizando renumeração sequencial dos promotores...
Total de contratados: 5


In [13]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple, List

def gerar_relatorio_final(df_solucao_final: pd.DataFrame,
                          lucro_final: float,
                          ampl_instance,
                          df_base_completo: pd.DataFrame,
                          timelimit_final: int = 10) -> str:
    """
    Roda o modelo uma última vez com a solução 100% travada
    para extrair todas as variáveis detalhadas (v, h, z),
    calcular tempos de trabalho e formatar o relatório final.

    """

    # Preparar dados para o AMPL

    # Todas as lojas com um promotor válido são "fixas"
    df_fixas = df_solucao_final[df_solucao_final['id_promotor_global'] > 0]
    lojas_fixas_ids_globais = list(df_fixas.index)

    mapa_global_para_local = {idx_global: local_id for local_id, idx_global
                                  in enumerate(df_base_completo.index, start=1)}
    mapa_local_para_global_reverso = {local: glob for glob, local in mapa_global_para_local.items()}

    M = len(df_base_completo)

    # Cálculo de tau (tempo de deslocamento)
    tau = 0.084694 * M + 9.938776

    N = int(df_solucao_final['id_promotor_global'].max())
    if np.isnan(N) or N <= 0: N = max(int(M / 4), 10)

    nome_arquivo_dat_FINAL = "dados_LNS_final_report.dat"

    #Preparação dos Dados
    df_base = df_base_completo.copy()

    # Renomear colunas para o padrão interno
    mapa_colunas = {
        'visit_duration_minutes': 'tempo_visita',
        'initial_frequency': 'freq_visita'
    }
    df_base.rename(columns=mapa_colunas, inplace=True)

    # Validação simples
    if 'tempo_visita' not in df_base.columns or 'freq_visita' not in df_base.columns:
        return "ERRO CRÍTICO: Colunas 'visit_duration_minutes' ou 'initial_frequency' não encontradas no DataFrame.", None, None

    # Garantir inteiros
    df_base['tempo_visita'] = df_base['tempo_visita'].fillna(0).astype(int)
    df_base['freq_visita'] = df_base['freq_visita'].fillna(0).astype(int)

    try:
        # Gerar o arquivo .dat com lojas travadas
        with open(nome_arquivo_dat_FINAL, 'w', encoding='utf-8') as f:
            f.write(f"# Arquivo de dados - Relatório Final (M={M}, N={N})\n\n")
            f.write(f"set PROMOTORES := {' '.join(str(i) for i in range(1, N + 1))};\n\n")
            f.write(f"set LOJAS := {' '.join(str(i) for i in range(1, M + 1))};\n\n")
            f.write(f"param M := {M};\n")
            f.write(f"param N := {N};\n\n")

            f.write("param X :=\n")
            for idx_global, row in df_base.iterrows():
                f.write(f"  {mapa_global_para_local[idx_global]}  {float(row['x_coordinate']):.6f}\n")
            f.write(";\n\n")

            f.write("param Y :=\n")
            for idx_global, row in df_base.iterrows():
                f.write(f"  {mapa_global_para_local[idx_global]}  {float(row['y_coordinate']):.6f}\n")
            f.write(";\n\n")

            f.write("param tempo_visita :=\n")
            for idx_global, row in df_base.iterrows():
                # Pega o valor real que veio do CSV
                f.write(f"  {mapa_global_para_local[idx_global]}  {int(row['tempo_visita'])}\n")
            f.write(";\n\n")

            # Matriz de lucro potencial
            lucro_cols = [f'profitability_freq_{f}' for f in range(1, 7)]

            f.write("# Lucro potencial (R$) para cada loja em funcao da frequencia (1 a 6 visitas)\n")
            f.write("param lucro_potencial: ")
            f.write(" ".join(str(f) for f in range(1, 7)))
            f.write(" :=\n")
            for idx_global, row in df_base.iterrows():
                idx_local = mapa_global_para_local[idx_global]
                line = f"  {idx_local} "
                line += " ".join(f"{row[col]:.2f}" for col in lucro_cols)
                f.write(line + "\n")
            f.write(";\n\n")

            # Frequência mínima
            f.write("param freq_minima :=\n")
            for idx_global, row in df_base.iterrows():
                f.write(f"  {mapa_global_para_local[idx_global]}  {int(row['freq_visita'])}\n")
            f.write(";\n\n")

            # DADOS DE FIXAÇÃO
            f.write("\n# --- DADOS DE FIXAÇÃO 100% (Relatório Final) ---\n")
            ids_fixas_locais = [mapa_global_para_local[idx] for idx in lojas_fixas_ids_globais]
            f.write(f"set LOJAS_FIXAS := {' '.join(map(str, ids_fixas_locais))};\n\n")

            f.write("param promotor_fixo :=\n")
            for idx_global in lojas_fixas_ids_globais:
                idx_local = mapa_global_para_local[idx_global]
                id_promotor = df_solucao_final.at[idx_global, 'id_promotor_global']
                f.write(f"  {idx_local}  {int(id_promotor)}\n")
            f.write(";\n")

    except Exception as e:
        return f"ERRO ao GERAR .dat final: {e}", None, None

    # Rodar o AMPL
    var_r = None
    var_c = None
    var_v = None
    var_h_df = None
    var_z = None

    try:
        ampl_commands = f"""
        reset;
        model modelo_promotores_estendido.mod;
        data {nome_arquivo_dat_FINAL};
        option solver gurobi;
        option gurobi_options 'mipgap=0.01';
        solve;
        """
        ampl_instance.eval(ampl_commands)

        # Extrair TODAS as variáveis necessárias
        var_r = ampl_instance.get_variable("r").get_values().to_pandas()
        var_c = ampl_instance.get_variable("c").get_values().to_pandas()
        var_v = ampl_instance.get_variable("v").get_values().to_pandas()
        var_h_df = ampl_instance.get_variable("h").get_values().to_pandas()
        var_z = ampl_instance.get_variable("z").get_values().to_pandas()


        # Função auxiliar para achar coluna de valor
        def get_val_col(df):
            for c in df.columns:
                if 'val' in c.lower(): return c
            return df.columns[0]

        col_r = get_val_col(var_r)
        col_c = get_val_col(var_c)
        col_v = get_val_col(var_v)
        col_h = get_val_col(var_h_df)

        promotores_usados = list(var_r[var_r[col_r] > 0.9].index.unique())
        atribuicoes = var_c[var_c[col_c] > 0.9]
        agenda = var_v[var_v[col_v] > 0.9]

    except Exception as e:
        return f"ERRO ao RESOLVER/EXTRAIR dados finais: {e}", None, None

    # RETORNAR var_v E var_z
    return var_v, var_z

In [14]:

if 'df_best' in locals() and 'lucro_best' in locals():
    # Chamar a função que acabamos de definir
    var_v, var_z = gerar_relatorio_final(
        df_solucao_final=df_best,
        lucro_final=lucro_best,
        ampl_instance=ampl,
        df_base_completo=df_completo_base
    )

    # CRIAR DICIONÁRIO DE FREQUÊNCIAS
    # var_z[j, f] = 1 se a loja j foi escolhida para frequência f
    freq_escolhida_por_loja = {}

    # Detectar coluna de valor
    col_z_val = [c for c in var_z.columns if 'val' in c.lower()][0]

    # Filtrar apenas z[j,f] = 1
    z_ativas = var_z[var_z[col_z_val] > 0.9]

    # Extrair frequência escolhida para cada loja
    for idx, row in z_ativas.iterrows():
        if isinstance(idx, tuple):
            j_local, f = idx
        else:
            # Caso seja índice simples (depende da versão amplpy)
            continue

        # Converter de ID local (1-based AMPL) para ID global (0-based pandas)
        j_global = j_local - 1
        freq_escolhida_por_loja[j_global] = int(f)

else:
    print("ERRO: 'df_best' ou 'lucro_best' não foram encontrados.")
    print("Por favor, certifique-se de que o loop LNS foi executado com sucesso.")

Gurobi 13.0.0:   mip:gap = 0.01
Gurobi 13.0.0: optimal solution; objective 6288.326234
519 simplex iterations
1 branching node
absmipgap=49.2038, relmipgap=0.00782462


#TSP

##Modelo AMPL

In [15]:
ampl_code_roteamento = """# ==============================================================================
# MODELO DE ROTEAMENTO (TSP CAMINHO ABERTO): Sem retorno ao início
# ==============================================================================

# --- 1. CONJUNTOS ---
set LOJAS_ROTA ordered;  # Lojas a visitar (ORDENADO para usar first/last)

# --- 2. PARÂMETROS ---
param n_lojas;
param X_rota{LOJAS_ROTA};
param Y_rota{LOJAS_ROTA};
param tipo_rota{LOJAS_ROTA} symbolic;

# Matriz de distâncias
param dist{i in LOJAS_ROTA, j in LOJAS_ROTA} :=
    sqrt((X_rota[i] - X_rota[j])^2 + (Y_rota[i] - Y_rota[j])^2);

# --- 3. VARIÁVEIS DE DECISÃO ---
var x{LOJAS_ROTA, LOJAS_ROTA} binary;  # x[i,j] = 1 se vai de i para j
var u{LOJAS_ROTA} >= 0, <= n_lojas;    # Ordem de visita (posição na rota)

# --- 4. FUNÇÃO OBJETIVO ---
# Minimizar a distância total percorrida (caminho aberto, sem retorno)
minimize distancia_total:
    sum{i in LOJAS_ROTA, j in LOJAS_ROTA: i <> j} dist[i,j] * x[i,j];

# --- 5. RESTRIÇÕES ---

# R1: Cada loja deve ter no máximo uma saída (última loja não tem saída)
subject to max_uma_saida{i in LOJAS_ROTA}:
    sum{j in LOJAS_ROTA: i <> j} x[i,j] <= 1;

# R2: Cada loja deve ter no máximo uma entrada (primeira loja não tem entrada)
subject to max_uma_entrada{j in LOJAS_ROTA}:
    sum{i in LOJAS_ROTA: i <> j} x[i,j] <= 1;

# R3: Total de arcos = n_lojas - 1 (caminho, não ciclo)
subject to total_arcos:
    sum{i in LOJAS_ROTA, j in LOJAS_ROTA: i <> j} x[i,j] = n_lojas - 1;

# R4: Conservação de fluxo - cada loja intermediária tem 1 entrada e 1 saída
# Exceto a primeira (só saída) e a última (só entrada)
subject to fluxo{k in LOJAS_ROTA}:
    sum{i in LOJAS_ROTA: i <> k} x[i,k] - sum{j in LOJAS_ROTA: j <> k} x[k,j] >= -1;

subject to fluxo2{k in LOJAS_ROTA}:
    sum{i in LOJAS_ROTA: i <> k} x[i,k] - sum{j in LOJAS_ROTA: j <> k} x[k,j] <= 1;

# R5: Eliminação de subciclos (MTZ adaptado para caminho)
subject to ordem_visita{i in LOJAS_ROTA, j in LOJAS_ROTA: i <> j}:
    u[i] - u[j] + n_lojas * x[i,j] <= n_lojas - 1;

# R6: Primeira loja visitada tem u >= 1
subject to ordem_minima{i in LOJAS_ROTA}:
    u[i] >= 1;

# R7: Última loja visitada tem u <= n_lojas
subject to ordem_maxima{i in LOJAS_ROTA}:
    u[i] <= n_lojas;
"""

# Salvar o código AMPL em um arquivo
with open('modelo_roteamento_tsp.mod', 'w') as f:
    f.write(ampl_code_roteamento)

##Programação Diária

In [16]:
# ==============================================================================
# TSP usando a programação diária do LNS
# ==============================================================================

from amplpy import AMPL
import pandas as pd


# Verificar se temos as variáveis necessárias
if 'df_best' not in locals() or 'df_completo_base' not in locals():
    print("ERRO: Variáveis 'df_best' ou 'df_completo_base' não encontradas.")
elif 'var_v' not in locals():
    print("ERRO: Variável 'var_v' não encontrada.")
else:

    # CRIAR MAPEAMENTO GLOBAL → LOCAL (0-19 → 1-20)
    mapa_global_para_local = {idx_global: local_id
                              for local_id, idx_global in enumerate(df_completo_base.index, start=1)}

    mapa_local_para_global = {v: k for k, v in mapa_global_para_local.items()}

    # Obter lista de promotores ativos
    promotores_ativos = sorted(df_best[df_best['id_promotor_global'] > 0]['id_promotor_global'].unique())

    # Detectar coluna de valor em var_v
    col_v_val = 'v.val'

    # Dias da semana
    dias_semana = {1: "Segunda-feira", 2: "Terça-feira", 3: "Quarta-feira",
                   4: "Quinta-feira", 5: "Sexta-feira", 6: "Sábado"}


    # Extrair programação diária do LNS
    programacao_LNS = {}

    for promotor_id in promotores_ativos:
        programacao_LNS[promotor_id] = {dia: [] for dia in range(1, 7)}

        # Lojas atribuídas ao promotor
        lojas_promotor_global = df_best[df_best['id_promotor_global'] == promotor_id].index.tolist()

        # Extrair visitas por dia da variável v[i,j,d]
        for dia in range(1, 7):
            for loja_global in lojas_promotor_global:
                # CONVERTER ID GLOBAL em LOCAL
                loja_local = mapa_global_para_local[loja_global]

                try:
                    # Acessar var_v usando IDs LOCAIS
                    mask = (var_v.index.get_level_values(0) == promotor_id) & \
                           (var_v.index.get_level_values(1) == loja_local) & \
                           (var_v.index.get_level_values(2) == dia)

                    if mask.any():
                        valor = var_v.loc[mask, col_v_val].values[0]
                        if valor > 0.5:
                            # Armazenar usando ID GLOBAL
                            programacao_LNS[promotor_id][dia].append(loja_global)
                except Exception as e:
                    pass

        total_visitas = sum(len(v) for v in programacao_LNS[promotor_id].values())


    # Aplicar TSP para cada promotor, para cada dia

    programacao_diaria_otimizada = {}

    for promotor_id in promotores_ativos:
        print(f"PROMOTOR {promotor_id}")

        programacao_diaria_otimizada[promotor_id] = {}

        for dia in range(1, 7):
            lojas_do_dia = programacao_LNS[promotor_id][dia]
            n_lojas = len(lojas_do_dia)

            if n_lojas == 0:
                programacao_diaria_otimizada[promotor_id][dia] = {
                    'lojas': [],
                    'rota_ordenada': [],
                    'distancia': 0
                }
                continue

            elif n_lojas == 1:
                programacao_diaria_otimizada[promotor_id][dia] = {
                    'lojas': lojas_do_dia,
                    'rota_ordenada': lojas_do_dia,
                    'distancia': 0
                }
                continue

            # Resolver TSP para as lojas do dia

            # Mapear IDs globais para IDs locais TSP
            id_map_tsp = {loja_id: i+1 for i, loja_id in enumerate(lojas_do_dia)}
            reverse_map_tsp = {v: k for k, v in id_map_tsp.items()}

            # Criar arquivo .dat para o TSP
            dat_filename = f'tsp_promotor{promotor_id}_dia{dia}.dat'

            with open(dat_filename, 'w') as f:
                f.write(f"# TSP - Promotor {promotor_id} - {dias_semana[dia]}\n\n")

                # Conjunto de lojas (IDs locais TSP)
                f.write("set LOJAS_ROTA := ")
                f.write(" ".join(str(id_map_tsp[l]) for l in lojas_do_dia))
                f.write(";\n\n")

                # Número de lojas
                f.write(f"param n_lojas := {n_lojas};\n\n")

                # Coordenadas X (IDs locais TSP)
                f.write("param X_rota :=\n")
                for loja_id in lojas_do_dia:
                    x_coord = df_completo_base.at[loja_id, 'x_coordinate']
                    f.write(f"   {id_map_tsp[loja_id]}   {x_coord:.2f}\n")
                f.write(";\n\n")

                # Coordenadas Y (IDs locais TSP)
                f.write("param Y_rota :=\n")
                for loja_id in lojas_do_dia:
                    y_coord = df_completo_base.at[loja_id, 'y_coordinate']
                    f.write(f"   {id_map_tsp[loja_id]}   {y_coord:.2f}\n")
                f.write(";\n")

                # Tipo da loja
                f.write("\nparam tipo_rota :=\n")
                for loja_id in lojas_do_dia:
                    tipo = df_completo_base.at[loja_id, 'type']
                    f.write(f"   {id_map_tsp[loja_id]}   {tipo}\n")
                f.write(";\n")

            # Resolver TSP
            ampl_tsp = AMPL()
            try:
                ampl_tsp.read('modelo_roteamento_tsp.mod')
                ampl_tsp.read_data(dat_filename)
                ampl_tsp.set_option('solver', 'gurobi')
                ampl_tsp.set_option('gurobi_options', 'timelimit=60')

                ampl_tsp.solve()

                solve_result = ampl_tsp.get_value('solve_result')

                if solve_result == 'solved' or 'limit' in solve_result:
                    distancia_total = ampl_tsp.get_objective('distancia_total').value()

                    # Extrair rota ordenada usando a variável u
                    x_tsp = ampl_tsp.get_variable('x').get_values().to_pandas()
                    u_tsp = ampl_tsp.get_variable('u').get_values().to_pandas()

                    col_x_val = [c for c in x_tsp.columns if 'val' in c.lower()][0]
                    col_u_val = [c for c in u_tsp.columns if 'val' in c.lower()][0]

                    # Usar variável u para ordenar
                    ordem_lojas = []
                    for loja_local in range(1, n_lojas + 1):
                        try:
                            ordem = u_tsp.loc[loja_local, col_u_val]
                            ordem_lojas.append((loja_local, ordem))
                        except:
                            ordem_lojas.append((loja_local, 999))

                    # Ordenar por u
                    ordem_lojas.sort(key=lambda x: x[1])
                    rota_local = [loja for loja, _ in ordem_lojas]

                    # Converter para IDs globais
                    rota = [reverse_map_tsp[id_local] for id_local in rota_local]

                    programacao_diaria_otimizada[promotor_id][dia] = {
                        'lojas': lojas_do_dia,
                        'rota_ordenada': rota,
                        'distancia': distancia_total
                    }

                else:
                    print(f"   Solução não ótima: {solve_result}")
                    print(f"   Usando ordem original: {' → '.join(map(str, lojas_do_dia))}\n")

                    programacao_diaria_otimizada[promotor_id][dia] = {
                        'lojas': lojas_do_dia,
                        'rota_ordenada': lojas_do_dia,
                        'distancia': None
                    }

            except Exception as e:
                print(f"   ERRO ao resolver TSP: {e}")
                print(f"   Usando ordem original: {' → '.join(map(str, lojas_do_dia))}\n")

                programacao_diaria_otimizada[promotor_id][dia] = {
                    'lojas': lojas_do_dia,
                    'rota_ordenada': lojas_do_dia,
                    'distancia': None
                }

            finally:
                ampl_tsp.close()

PROMOTOR 1
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 531.3350126
9 simplex iterations
1 branching node
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 363.6862293
27 simplex iterations
1 branching node
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 531.3350126
9 simplex iterations
1 branching node
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 531.3350126
9 simplex iterations
1 branching node
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 363.6862293
27 simplex iterations
1 branching node
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 221.7843119
0 simplex iterations
PROMOTOR 2
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 167.2513609
9 simplex iterations
1 branching node
Gurobi 13.0.0:   lim:time = 60
Gurobi 13.0.0: optimal solution; objective 167.2513609
9 simplex iterations
1 branching node
G

In [17]:
# ==============================================================================
# CÁLCULO DO LUCRO FINAL COM ROTAS TSP OTIMIZADAS
# ==============================================================================

from datetime import datetime
import numpy as np


# --- Parâmetros de custo ---
P_rep = 750     # Custo fixo de contratar um promotor
P_dist = 0.06   # Custo por unidade de distância
P_he = 20.45    # Custo por minuto de hora extra
PB = 5.0        # Penalidade por desbalanceamento

M = len(df_completo_base)
tau = 0.084694 * M + 9.938776

# Número de promotores
num_promotores = len(promotores_ativos)
custo_promotores = P_rep * num_promotores

# --- Calcular receita total  ---

receita_total = 0
for loja_id in df_completo_base.index:
    freq = df_completo_base.at[loja_id, 'initial_frequency']
    # Buscar o lucro correspondente à frequência
    col_lucro = f'profitability_freq_{int(freq)}'
    if col_lucro in df_completo_base.columns:
        receita_total += df_completo_base.at[loja_id, col_lucro]
    else:
        receita_total += df_completo_base.at[loja_id, 'baseline_profitability']

# --- Calcular distância total ---
distancia_total_otimizada = 0
for promotor_id, dias_prog in programacao_diaria_otimizada.items():
    for dia, prog in dias_prog.items():
        if prog['distancia'] is not None:
            distancia_total_otimizada += prog['distancia']

custo_distancia_otimizada = P_dist * distancia_total_otimizada

# --- Calcular horas extras ---
custo_horas_extras = 0
if 'var_h_df' in dir() and var_h_df is not None:
    col_h = [c for c in var_h_df.columns if 'val' in c.lower()][0]
    total_he = var_h_df[col_h].sum()
    custo_horas_extras = P_he * total_he
else:
    print(f"HORAS EXTRAS: Não disponível")

# --- Calcular penalidade de desbalanceamento ---
cargas_promotores = {}
for promotor_id in promotores_ativos:
    carga = 0
    for dia in range(1, 7):
        prog = programacao_diaria_otimizada.get(promotor_id, {}).get(dia, {})
        for loja_id in prog.get('rota_ordenada', []):
            tempo_visita = df_completo_base.at[loja_id, 'visit_duration_minutes']
            carga += tempo_visita + tau
    cargas_promotores[promotor_id] = carga

if cargas_promotores:
    carga_max = max(cargas_promotores.values())
    carga_min = min(cargas_promotores.values())
    custo_desbalanceamento = PB * (carga_max - carga_min)
else:
    carga_max = carga_min = 0
    custo_desbalanceamento = 0

# --- LUCRO FINAL ---
lucro_final_otimizado = receita_total - custo_promotores - custo_distancia_otimizada - custo_horas_extras - custo_desbalanceamento

print(f"\nRECEITAS:")
print(f"   (+) Receita total das lojas:          R$ {receita_total:>12,.2f}")
print(f"\nCUSTOS:")
print(f"   (-) Custo de promotores ({num_promotores} x R$750):  R$ {custo_promotores:>12,.2f}")
print(f"   (-) Custo de distância:               R$ {custo_distancia_otimizada:>12,.2f}")
print(f"   (-) Custo de horas extras:            R$ {custo_horas_extras:>12,.2f}")
print(f"   (-) Penalidade desbalanceamento:      R$ {custo_desbalanceamento:>12,.2f}")
print(f"       (CargaMax: {carga_max:.1f} min, CargaMin: {carga_min:.1f} min)")
print(f"   ─────────────────────────────────────────────────────────")
print(f"   (=) LUCRO FINAL:            R$ {lucro_final_otimizado:>12,.2f}")

# Comparar com o lucro do LNS (sem TSP otimizado)
if 'lucro_best' in dir():
    diferenca = lucro_final_otimizado - lucro_best
    print(f"\n COMPARAÇÃO:")
    print(f"   Lucro LNS:        R$ {lucro_best:>12,.2f}")
    print(f"   Lucro Final:      R$ {lucro_final_otimizado:>12,.2f}")
    print(f"   Diferença:        R$ {diferenca:>12,.2f}")

# Salvar em variáveis globais para uso posterior
lucro_ampl_original = lucro_best if 'lucro_best' in dir() else 0

HORAS EXTRAS: Não disponível

RECEITAS:
   (+) Receita total das lojas:          R$     7,870.00

CUSTOS:
   (-) Custo de promotores (5 x R$750):  R$     3,750.00
   (-) Custo de distância:         R$       412.03
   (-) Custo de horas extras:            R$         0.00
   (-) Penalidade desbalanceamento:      R$        59.18
       (CargaMax: 2487.8 min, CargaMin: 2475.9 min)
   ─────────────────────────────────────────────────────────
   (=) LUCRO FINAL:            R$     3,648.79

 COMPARAÇÃO:
   Lucro LNS:        R$     6,299.33
   Lucro Final:                R$     3,648.79
   Diferença:                            R$    -2,650.54


In [18]:
# ==============================================================================
# RELATÓRIO FINAL
# ==============================================================================

arquivo_relatorio = "relatorio_final_completo.txt"

with open(arquivo_relatorio, 'w', encoding='utf-8') as f:
    f.write("=" * 80 + "\n")
    f.write("       RELATÓRIO FINAL - OTIMIZAÇÃO DE PROMOTORES E ROTAS (HEURÍSTICA)\n")
    f.write("=" * 80 + "\n")
    f.write(f"Data de geração: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    # === RESUMO EXECUTIVO ===
    f.write("=" * 80 + "\n")
    f.write("                         RESUMO EXECUTIVO\n")
    f.write("=" * 80 + "\n\n")

    f.write("┌─────────────────────────────────────────────────────────────────┐\n")
    f.write(f"│ LUCRO SEM ROTA OTIMIZADA (LNS):      R$ {lucro_best:>15,.2f}      │\n")
    f.write(f"│ LUCRO COM ROTA OTIMIZADA (TSP):      R$ {lucro_final_otimizado:>15,.2f}      │\n")
    f.write(f"│ DIFERENÇA:                           R$ {lucro_final_otimizado - lucro_best:>15,.2f}      │\n")
    f.write("└─────────────────────────────────────────────────────────────────┘\n\n")

    # === DETALHAMENTO DE RECEITAS E CUSTOS ===
    f.write("─" * 80 + "\n")
    f.write("DETALHAMENTO DE RECEITAS E CUSTOS\n")
    f.write("─" * 80 + "\n\n")

    f.write("RECEITAS:\n")
    f.write(f"   (+) Receita total das lojas:          R$ {receita_total:>12,.2f}\n\n")

    f.write("CUSTOS:\n")
    f.write(f"   (-) Custo de promotores ({num_promotores} x R$750):  R$ {custo_promotores:>12,.2f}\n")
    f.write(f"   (-) Custo de distância (TSP):         R$ {custo_distancia_otimizada:>12,.2f}\n")
    f.write(f"   (-) Custo de horas extras:            R$ {custo_horas_extras:>12,.2f}\n")
    f.write(f"   (-) Penalidade desbalanceamento:      R$ {custo_desbalanceamento:>12,.2f}\n")
    f.write(f"       (CargaMax: {carga_max:.1f} min, CargaMin: {carga_min:.1f} min, Diferenca: {carga_max - carga_min:.1f} min)\n")
    f.write(f"   ─────────────────────────────────────────────────────────\n")
    f.write(f"   (=) LUCRO FINAL:                      R$ {lucro_final_otimizado:>12,.2f}\n\n")

    # === INDICADORES ===
    f.write("─" * 80 + "\n")
    f.write("INDICADORES OPERACIONAIS\n")
    f.write("─" * 80 + "\n\n")

    f.write(f"   Número de lojas:                      {M}\n")
    f.write(f"   Número de promotores:                 {num_promotores}\n")
    f.write(f"   Média de lojas por promotor:          {M/num_promotores:.1f}\n")
    f.write(f"   Tempo médio de deslocamento:          {tau:.2f} min\n")
    f.write(f"   Distância total (TSP):                {distancia_total_otimizada:.2f} unidades\n\n")
    f.write(f"   Carga maxima (promotor):              {carga_max:.2f} min\n")
    f.write(f"   Carga minima (promotor):              {carga_min:.2f} min\n")
    f.write(f"   Desbalanceamento de carga:            {carga_max - carga_min:.2f} min\n\n")

    # === DETALHAMENTO POR PROMOTOR ===
    f.write("=" * 80 + "\n")
    f.write("                    DETALHAMENTO POR PROMOTOR\n")
    f.write("=" * 80 + "\n\n")

    dias_semana = {1: "Segunda", 2: "Terça", 3: "Quarta", 4: "Quinta", 5: "Sexta", 6: "Sábado"}

    for promotor_id in sorted(promotores_ativos):
        lojas_prom = df_best[df_best['id_promotor_global'] == promotor_id].index.tolist()

        # Calcular métricas
        visitas_total = sum(len(programacao_diaria_otimizada[promotor_id][d]['rota_ordenada'])
                          for d in range(1, 7))
        dist_prom = sum(programacao_diaria_otimizada[promotor_id][d].get('distancia', 0) or 0
                       for d in range(1, 7))


        # Calcular horas extras e receita do promotor
        horas_extras_prom = 0  # Não disponível diretamente, usar 0 ou extrair de var_h_df se disponível
        receita_prom = sum(
            df_completo_base.at[loja_id, f'profitability_freq_{freq_escolhida_por_loja.get(loja_id, df_completo_base.at[loja_id, "initial_frequency"])}']
            for loja_id in lojas_prom
            if f'profitability_freq_{freq_escolhida_por_loja.get(loja_id, df_completo_base.at[loja_id, "initial_frequency"])}' in df_completo_base.columns
        )
        f.write(f"┌{'─'*78}┐\n")
        f.write(f"│ PROMOTOR {promotor_id}│\n")
        f.write(f"│ Distancia otimizada: {dist_prom:.2f} un│\n")
        f.write(f"├{'─'*78}┤\n")
        lojas_prom_display = [l+1 for l in lojas_prom]  # Converter para 1-based
        f.write(f"│ Lojas atribuídas: {str(lojas_prom_display):<59}│\n")
        f.write(f"│ Horas extras: {horas_extras_prom:.1f} min│\n")
        f.write(f"│ Receita das lojas: R$ {receita_prom:,.2f}│\n")
        f.write(f"│ Total de visitas/semana: {visitas_total:<52}│\n")
        f.write(f"│ Distância otimizada: {dist_prom:.2f} un{' '*52}│\n")
        f.write(f"└{'─'*78}┘\n\n")

        # Agenda semanal com rotas otimizadas
        f.write("   AGENDA SEMANAL (rotas otimizadas):\n")
        for dia in range(1, 7):
            prog = programacao_diaria_otimizada[promotor_id][dia]
            rota = prog.get('rota_ordenada', [])
            dist = prog.get('distancia', 0) or 0

            if rota:
                rota_str = ' → '.join([f"Loja {l+1}" for l in rota])
                f.write(f"   {dias_semana[dia]:>10}: {rota_str} ({dist:.1f} un)\n")
            else:
                f.write(f"   {dias_semana[dia]:>10}: (dia livre)\n")
        f.write("\n")


    f.write("=" * 80 + "\n")
    f.write("                      DETALHAMENTO POR LOJA\n")
    f.write("=" * 80 + "\n\n")

    f.write(f"{'Loja':<6}{'Tipo':<6}{'Promotor':<10}{'Freq Min':<10}{'Freq Esc':<10}{'Lucro (R$)':<12}{'Status':<15}\n")
    f.write("-" * 80 + "\n")

    for loja_id in df_completo_base.index:
        tipo = df_completo_base.at[loja_id, 'type']
        promotor = df_best.at[loja_id, 'id_promotor_global'] if loja_id in df_best.index else -1
        freq_min = df_completo_base.at[loja_id, 'initial_frequency']
        freq_esc = freq_escolhida_por_loja.get(loja_id, freq_min)

        col_lucro = f'profitability_freq_{int(freq_esc)}'
        lucro = df_completo_base.at[loja_id, col_lucro] if col_lucro in df_completo_base.columns else 0

        status = "= MINIMA" if freq_esc == freq_min else f"ACIMA (+{freq_esc - freq_min})"

        f.write(f"{loja_id+1:<6}{tipo:<6}{int(promotor):<10}{freq_min:<10}{freq_esc:<10}{lucro:<12,.2f}{status:<15}\n")

    f.write("-" * 80 + "\n")
    f.write(f"{'TOTAL':<42}{'':<10}{receita_total:<12,.2f}\n\n")

    f.write("=" * 80 + "\n")
    f.write("                    ANALISE DE FREQUENCIAS\n")
    f.write("=" * 80 + "\n\n")

    lojas_freq_minima = sum(1 for loja_id in df_completo_base.index
                            if freq_escolhida_por_loja.get(loja_id, df_completo_base.at[loja_id, 'initial_frequency'])
                               == df_completo_base.at[loja_id, 'initial_frequency'])
    lojas_acima_minima = M - lojas_freq_minima

    f.write(f"   Lojas na frequencia minima:    {lojas_freq_minima} ({lojas_freq_minima/M*100:.1f}%)\n")
    f.write(f"   Lojas acima da freq minima:    {lojas_acima_minima} ({lojas_acima_minima/M*100:.1f}%)\n")
    f.write(f"   Total de lojas:                {M}\n\n")

print(f"Relatório salvo em: {arquivo_relatorio}")

Relatório salvo em: relatorio_final_completo.txt


In [19]:
# ==============================================================================
# RELATÓRIO EXCEL COM VALIDAÇÕES
# ==============================================================================

import pandas as pd
import numpy as np

arquivo_excel = "resultado_otimizacao_heuristica.xlsx"

# var_z[j, f] = 1 se a loja j foi escolhida para frequência f
freq_escolhida_por_loja = {}

if 'var_z' in dir() and var_z is not None:
    # Detectar coluna de valor
    col_z_val = [c for c in var_z.columns if 'val' in c.lower()][0]

    # Filtrar apenas z[j,f] = 1
    z_ativas = var_z[var_z[col_z_val] > 0.9]

    # Extrair frequência escolhida para cada loja
    for idx, row in z_ativas.iterrows():
        if isinstance(idx, tuple):
            j_local, f = idx
        else:
            continue

        # Converter de ID local (1-based AMPL) para ID global (0-based pandas)
        j_global = j_local - 1
        freq_escolhida_por_loja[j_global] = int(f)
else:
    print("\n var_z não disponível, usando initial_frequency como fallback")

# Calcular custo de deslocamento POR LOJA
def calcular_distancia(loja1, loja2):
    x1, y1 = df_completo_base.at[loja1, 'x_coordinate'], df_completo_base.at[loja1, 'y_coordinate']
    x2, y2 = df_completo_base.at[loja2, 'x_coordinate'], df_completo_base.at[loja2, 'y_coordinate']
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

# Inicializar custo de deslocamento por loja
custo_deslocamento_por_loja = {loja: 0.0 for loja in df_completo_base.index}
distancia_por_loja = {loja: 0.0 for loja in df_completo_base.index}

# Percorrer todas as rotas otimizadas
for promotor_id in promotores_ativos:
    for dia in range(1, 7):
        prog = programacao_diaria_otimizada[promotor_id][dia]
        rota = prog.get('rota_ordenada', [])

        if len(rota) >= 2:
            for i in range(len(rota) - 1):
                loja_origem = rota[i]
                loja_destino = rota[i + 1]
                dist = calcular_distancia(loja_origem, loja_destino)
                distancia_por_loja[loja_destino] += dist
                custo_deslocamento_por_loja[loja_destino] += P_dist * dist

# Criar DataFrame do relatório por loja
lista_relatorio = []

for loja_id in df_completo_base.index:
    # Encontrar promotor
    promotor_loja = df_best.at[loja_id, 'id_promotor_global'] if loja_id in df_best.index else -1

    # Usar freq_escolhida_por_loja se disponível
    freq = freq_escolhida_por_loja.get(loja_id, df_completo_base.at[loja_id, 'initial_frequency'])

    # Receita
    col_lucro = f'profitability_freq_{int(freq)}'
    receita_loja = df_completo_base.at[loja_id, col_lucro] if col_lucro in df_completo_base.columns else 0

    # Custo de deslocamento
    custo_desloc = custo_deslocamento_por_loja[loja_id]
    distancia_loja = distancia_por_loja[loja_id]

    # Lucro resultante
    lucro_resultante = receita_loja - custo_desloc

    lista_relatorio.append({
        'Loja_ID': loja_id + 1,  # +1 para exibir 1-based
        'Tipo': df_completo_base.at[loja_id, 'type'],
        'Promotor_Alocado': int(promotor_loja) if promotor_loja > 0 else '-',
        'Freq_Escolhida': int(freq),
        'Distancia_Total_Un': round(distancia_loja, 2),
        'Custo_Deslocamento_R$': round(custo_desloc, 2),
        'Receita_R$': round(receita_loja, 2),
        'Lucro_Resultante_R$': round(lucro_resultante, 2)
    })

df_relatorio = pd.DataFrame(lista_relatorio)

# Linha de TOTAL
total_row = {
    'Loja_ID': 'TOTAL',
    'Tipo': '-',
    'Promotor_Alocado': '-',
    'Freq_Escolhida': '-',
    'Distancia_Total_Un': round(df_relatorio['Distancia_Total_Un'].sum(), 2),
    'Custo_Deslocamento_R$': round(df_relatorio['Custo_Deslocamento_R$'].sum(), 2),
    'Receita_R$': round(df_relatorio['Receita_R$'].sum(), 2),
    'Lucro_Resultante_R$': round(df_relatorio['Lucro_Resultante_R$'].sum(), 2)
}

df_relatorio = pd.concat([df_relatorio, pd.DataFrame([total_row])], ignore_index=True)

# Salvar Excel
with pd.ExcelWriter(arquivo_excel, engine='openpyxl') as writer:

    df_relatorio.to_excel(writer, sheet_name='Relatorio_Por_Loja', index=False)

    erros_encontrados = 0
    resumo_validacao = []

    # Obter coordenadas
    X_dict = {idx: df_completo_base.at[idx, 'x_coordinate'] for idx in df_completo_base.index}
    Y_dict = {idx: df_completo_base.at[idx, 'y_coordinate'] for idx in df_completo_base.index}

    # Jornada e dias
    jornada_diaria = {1: 480, 2: 480, 3: 480, 4: 480, 5: 480, 6: 240}
    dias_semana = {1: "Segunda", 2: "Terça", 3: "Quarta", 4: "Quinta", 5: "Sexta", 6: "Sábado"}

    # VALIDAÇÃO 1: JORNADA DIÁRIA
    validacao_jornada = []

    for promotor_id in promotores_ativos:
        for dia in range(1, 7):
            prog = programacao_diaria_otimizada.get(promotor_id, {}).get(dia, {})
            lojas_dia = prog.get('rota_ordenada', [])

            tempo_visitas = sum(df_completo_base.at[loja, 'visit_duration_minutes'] for loja in lojas_dia) if lojas_dia else 0
            tempo_deslocamento = len(lojas_dia) * tau if lojas_dia else 0
            tempo_total = tempo_visitas + tempo_deslocamento

            jornada_permitida = jornada_diaria[dia]
            horas_extras_dia = 0  # Não disponível diretamente no heurística
            limite_com_he = jornada_permitida + horas_extras_dia

            dentro_limite = tempo_total <= limite_com_he + 0.01
            status = "OK" if dentro_limite else "VIOLAÇÃO"
            if not dentro_limite:
                erros_encontrados += 1

            validacao_jornada.append({
                'Promotor': promotor_id,
                'Dia_Num': dia,
                'Dia_Nome': dias_semana[dia],
                'Num_Visitas': len(lojas_dia),
                'Tempo_Visitas_Min': round(tempo_visitas, 1),
                'Tempo_Deslocamento_Min': round(tempo_deslocamento, 1),
                'Tempo_Total_Min': round(tempo_total, 1),
                'Jornada_Min': jornada_permitida,
                'Horas_Extras_Min': round(horas_extras_dia, 1),
                'Limite_Total_Min': round(limite_com_he, 1),
                'Folga_Min': round(limite_com_he - tempo_total, 1),
                'Utilizacao_%': round(tempo_total / jornada_permitida * 100, 1) if jornada_permitida > 0 else 0,
                'Status': status
            })

    df_val_jornada = pd.DataFrame(validacao_jornada)
    df_val_jornada.to_excel(writer, sheet_name='Val_Jornada_Diaria', index=False)

    ok_jornada = len(df_val_jornada[df_val_jornada['Status'] == 'OK'])
    total_jornada = len(df_val_jornada)
    resumo_validacao.append({
        'Restricao': 'Jornada Diária',
        'Descricao': 'Tempo total ≤ Jornada + Horas Extras',
        'Total_Verificacoes': total_jornada,
        'OK': ok_jornada,
        'Violacoes': total_jornada - ok_jornada,
        'Percentual_OK': f"{ok_jornada/total_jornada*100:.1f}%"
    })

    # VALIDAÇÃO 2: LIMITE DE CARTEIRA (MAX LOJAS POR PROMOTOR)
    validacao_carteira = []
    max_lojas = 8

    for promotor_id in promotores_ativos:
        lojas_promotor = df_best[df_best['id_promotor_global'] == promotor_id].index.tolist()
        num_lojas = len(lojas_promotor)

        dentro_limite = num_lojas <= max_lojas
        status = "OK" if dentro_limite else "VIOLAÇÃO"
        if not dentro_limite:
            erros_encontrados += 1

        validacao_carteira.append({
            'Promotor': promotor_id,
            'Num_Lojas': num_lojas,
            'Limite_Max': max_lojas,
            'Folga': max_lojas - num_lojas,
            'Lojas_Atribuidas': ', '.join(map(str, [l + 1 for l in lojas_promotor])),  # +1 para 1-based
            'Status': status
        })

    df_val_carteira = pd.DataFrame(validacao_carteira)
    df_val_carteira.to_excel(writer, sheet_name='Val_Limite_Carteira', index=False)

    ok_carteira = len(df_val_carteira[df_val_carteira['Status'] == 'OK'])
    total_carteira = len(df_val_carteira)
    resumo_validacao.append({
        'Restricao': 'Limite de Carteira',
        'Descricao': 'Cada promotor tem ≤ 8 lojas',
        'Total_Verificacoes': total_carteira,
        'OK': ok_carteira,
        'Violacoes': total_carteira - ok_carteira,
        'Percentual_OK': f"{ok_carteira/total_carteira*100:.1f}%"
    })

    # VALIDAÇÃO 3: FREQUÊNCIA MÍNIMA RESPEITADA
    validacao_frequencia = []

    for loja_id in df_completo_base.index:
        freq_min = df_completo_base.at[loja_id, 'initial_frequency']
        freq_esc = freq_escolhida_por_loja.get(loja_id, freq_min)

        dentro_limite = freq_esc >= freq_min
        status = "OK" if dentro_limite else "VIOLAÇÃO"
        if not dentro_limite:
            erros_encontrados += 1

        promotor_loja = df_best.at[loja_id, 'id_promotor_global'] if loja_id in df_best.index else -1

        validacao_frequencia.append({
            'Loja_ID': loja_id + 1,  # +1 para 1-based
            'Tipo': df_completo_base.at[loja_id, 'type'],
            'Promotor': int(promotor_loja) if promotor_loja > 0 else '-',
            'Freq_Minima': freq_min,
            'Freq_Escolhida': freq_esc,
            'Diferenca': freq_esc - freq_min,
            'Status_Freq': 'MÍNIMA' if freq_esc == freq_min else f'ACIMA (+{freq_esc - freq_min})',
            'Status': status
        })

    df_val_frequencia = pd.DataFrame(validacao_frequencia)
    df_val_frequencia.to_excel(writer, sheet_name='Val_Frequencia_Min', index=False)

    ok_frequencia = len(df_val_frequencia[df_val_frequencia['Status'] == 'OK'])
    total_frequencia = len(df_val_frequencia)
    resumo_validacao.append({
        'Restricao': 'Frequência Mínima',
        'Descricao': 'Freq escolhida ≥ Freq mínima',
        'Total_Verificacoes': total_frequencia,
        'OK': ok_frequencia,
        'Violacoes': total_frequencia - ok_frequencia,
        'Percentual_OK': f"{ok_frequencia/total_frequencia*100:.1f}%"
    })

    # VALIDAÇÃO 4: ALOCAÇÃO ÚNICA (CADA LOJA TEM 1 PROMOTOR)
    validacao_alocacao = []

    for loja_id in df_completo_base.index:
        promotor_loja = df_best.at[loja_id, 'id_promotor_global'] if loja_id in df_best.index else -1
        num_promotores = 1 if promotor_loja > 0 else 0

        correto = num_promotores == 1
        status = "OK" if correto else "VIOLAÇÃO"
        if not correto:
            erros_encontrados += 1

        validacao_alocacao.append({
            'Loja_ID': loja_id + 1,  # +1 para 1-based
            'Tipo': df_completo_base.at[loja_id, 'type'],
            'Num_Promotores_Alocados': num_promotores,
            'Esperado': 1,
            'Promotores': str(int(promotor_loja)) if promotor_loja > 0 else 'NENHUM',
            'Status': status
        })

    df_val_alocacao = pd.DataFrame(validacao_alocacao)
    df_val_alocacao.to_excel(writer, sheet_name='Val_Alocacao_Unica', index=False)

    ok_alocacao = len(df_val_alocacao[df_val_alocacao['Status'] == 'OK'])
    total_alocacao = len(df_val_alocacao)
    resumo_validacao.append({
        'Restricao': 'Alocação Única',
        'Descricao': 'Cada loja tem exatamente 1 promotor',
        'Total_Verificacoes': total_alocacao,
        'OK': ok_alocacao,
        'Violacoes': total_alocacao - ok_alocacao,
        'Percentual_OK': f"{ok_alocacao/total_alocacao*100:.1f}%"
    })

    # VALIDAÇÃO 5: CONSISTÊNCIA DE VISITAS
    validacao_consistencia = []

    for loja_id in df_completo_base.index:
        # Usar a frequência extraída de var_z
        freq_esc = freq_escolhida_por_loja.get(loja_id, df_completo_base.at[loja_id, 'initial_frequency'])

        # Contar visitas reais na programação
        visitas_reais = 0
        dias_visitados = []
        for promotor_id in promotores_ativos:
            for dia in range(1, 7):
                prog = programacao_diaria_otimizada.get(promotor_id, {}).get(dia, {})
                lojas_dia = prog.get('rota_ordenada', [])
                if loja_id in lojas_dia:
                    visitas_reais += 1
                    dias_visitados.append(dias_semana[dia])

        # Validação: visitas reais devem ser igual à frequência escolhida
        correto = visitas_reais == freq_esc
        status = "OK" if correto else "VIOLAÇÃO"
        if not correto:
            erros_encontrados += 1

        promotor_loja = df_best.at[loja_id, 'id_promotor_global'] if loja_id in df_best.index else -1

        validacao_consistencia.append({
            'Loja_ID': loja_id + 1,  # +1 para 1-based
            'Tipo': df_completo_base.at[loja_id, 'type'],
            'Promotor': int(promotor_loja) if promotor_loja > 0 else '-',
            'Freq_Escolhida': freq_esc,
            'Visitas_Reais': visitas_reais,
            'Diferenca': visitas_reais - freq_esc,
            'Dias_Visitados': ', '.join(dias_visitados),
            'Status': status
        })

    df_val_consistencia = pd.DataFrame(validacao_consistencia)
    df_val_consistencia.to_excel(writer, sheet_name='Val_Consistencia_Visitas', index=False)

    ok_consistencia = len(df_val_consistencia[df_val_consistencia['Status'] == 'OK'])
    total_consistencia = len(df_val_consistencia)
    resumo_validacao.append({
        'Restricao': 'Consistência de Visitas',
        'Descricao': 'Total visitas = Frequência escolhida',
        'Total_Verificacoes': total_consistencia,
        'OK': ok_consistencia,
        'Violacoes': total_consistencia - ok_consistencia,
        'Percentual_OK': f"{ok_consistencia/total_consistencia*100:.1f}%"
    })


    # VALIDAÇÃO 6: VISITA APENAS PELO PROMOTOR RESPONSÁVEL
    validacao_dono = []

    for loja_id in df_completo_base.index:
        promotor_responsavel = df_best.at[loja_id, 'id_promotor_global'] if loja_id in df_best.index else -1

        # Verificar quais promotores visitaram esta loja
        promotores_que_visitaram = {}
        for promotor_id in promotores_ativos:
            visitas_promotor = 0
            for dia in range(1, 7):
                prog = programacao_diaria_otimizada.get(promotor_id, {}).get(dia, {})
                lojas_dia = prog.get('rota_ordenada', [])
                if loja_id in lojas_dia:
                    visitas_promotor += 1
            if visitas_promotor > 0:
                promotores_que_visitaram[promotor_id] = visitas_promotor

        # Verificar se apenas o responsável visitou
        apenas_responsavel = (
            len(promotores_que_visitaram) == 0 or
            (len(promotores_que_visitaram) == 1 and promotor_responsavel in promotores_que_visitaram)
        )
        status = "OK" if apenas_responsavel else "VIOLAÇÃO"
        if not apenas_responsavel:
            erros_encontrados += 1

        validacao_dono.append({
            'Loja_ID': loja_id + 1,  # +1 para 1-based
            'Tipo': df_completo_base.at[loja_id, 'type'],
            'Promotor_Responsavel': int(promotor_responsavel) if promotor_responsavel > 0 else '-',
            'Promotores_Visitaram': ', '.join([f"P{p}({v}x)" for p, v in promotores_que_visitaram.items()]),
            'Num_Promotores_Visitaram': len(promotores_que_visitaram),
            'Status': status
        })

    df_val_dono = pd.DataFrame(validacao_dono)
    df_val_dono.to_excel(writer, sheet_name='Val_Visita_Pelo_Dono', index=False)

    ok_dono = len(df_val_dono[df_val_dono['Status'] == 'OK'])
    total_dono = len(df_val_dono)
    resumo_validacao.append({
        'Restricao': 'Visita pelo Dono',
        'Descricao': 'Só o promotor responsável visita a loja',
        'Total_Verificacoes': total_dono,
        'OK': ok_dono,
        'Violacoes': total_dono - ok_dono,
        'Percentual_OK': f"{ok_dono/total_dono*100:.1f}%"
    })

    # VALIDAÇÃO 7: VERIFICAÇÃO DE DISTÂNCIAS (TSP vs CALCULADO)
    validacao_distancia = []

    for promotor_id in promotores_ativos:
        for dia in range(1, 7):
            prog = programacao_diaria_otimizada.get(promotor_id, {}).get(dia, {})
            rota = prog.get('rota_ordenada', [])
            dist_tsp = prog.get('distancia', 0) or 0

            # Calcular distância manualmente
            dist_calculada = 0
            if len(rota) >= 2:
                for i in range(len(rota) - 1):
                    l1, l2 = rota[i], rota[i+1]
                    d = np.sqrt((X_dict[l1] - X_dict[l2])**2 + (Y_dict[l1] - Y_dict[l2])**2)
                    dist_calculada += d

            diferenca = abs(dist_tsp - dist_calculada)
            correto = diferenca < 0.1  # Tolerância de 0.1
            status = "OK" if correto else "DIFERENÇA"
            if not correto:
                erros_encontrados += 1

            validacao_distancia.append({
                'Promotor': promotor_id,
                'Dia_Num': dia,
                'Dia_Nome': dias_semana[dia],
                'Num_Lojas': len(rota),
                'Rota': ' → '.join(map(str, [l + 1 for l in rota])) if rota else '-',  # +1 para 1-based
                'Dist_TSP': round(dist_tsp, 2),
                'Dist_Calculada': round(dist_calculada, 2),
                'Diferenca': round(diferenca, 2),
                'Status': status
            })

    df_val_distancia = pd.DataFrame(validacao_distancia)
    df_val_distancia.to_excel(writer, sheet_name='Val_Distancias', index=False)

    ok_distancia = len(df_val_distancia[df_val_distancia['Status'] == 'OK'])
    total_distancia = len(df_val_distancia)
    resumo_validacao.append({
        'Restricao': 'Distâncias TSP',
        'Descricao': 'Dist TSP = Dist calculada manualmente',
        'Total_Verificacoes': total_distancia,
        'OK': ok_distancia,
        'Violacoes': total_distancia - ok_distancia,
        'Percentual_OK': f"{ok_distancia/total_distancia*100:.1f}%"
    })

    # ABA RESUMO DE VALIDAÇÃO
    total_verificacoes = sum(r['Total_Verificacoes'] for r in resumo_validacao)
    total_ok = sum(r['OK'] for r in resumo_validacao)
    total_violacoes = sum(r['Violacoes'] for r in resumo_validacao)

    resumo_validacao.append({
        'Restricao': 'TOTAL GERAL',
        'Descricao': '-',
        'Total_Verificacoes': total_verificacoes,
        'OK': total_ok,
        'Violacoes': total_violacoes,
        'Percentual_OK': f"{total_ok/total_verificacoes*100:.1f}%"
    })

    df_resumo_val = pd.DataFrame(resumo_validacao)
    df_resumo_val.to_excel(writer, sheet_name='Resumo_Validacao', index=False)

    # ABA: DETALHAMENTO DIÁRIO POR PROMOTOR
    detalhamento_diario = []

    # Lista de todas as lojas
    todas_lojas = list(df_completo_base.index)

    for promotor_id in promotores_ativos:
        for dia in range(1, 7):
            # Obter dados da programação otimizada
            prog = programacao_diaria_otimizada.get(promotor_id, {}).get(dia, {})
            rota = prog.get('rota_ordenada', [])
            distancia_total_rota = prog.get('distancia', 0) or 0

            # Calcular tempos
            tempo_visitas = sum(df_completo_base.at[loja, 'visit_duration_minutes']
                               for loja in rota) if rota else 0
            tempo_deslocamento = len(rota) * tau if rota else 0
            tempo_total = tempo_visitas + tempo_deslocamento

            jornada = jornada_diaria[dia]
            horas_extras = 0

            # Criar dicionário base
            linha = {
                'Promotor': promotor_id,
                'Dia_Num': dia,
                'Dia_Nome': dias_semana[dia],
                'Num_Visitas': len(rota),
                'Lojas_Visitadas': ', '.join(map(str, [l + 1 for l in rota])),  # +1 para 1-based
                'Rota_Otimizada': ' → '.join(map(str, [l + 1 for l in rota])) if rota else '-',  # +1 para 1-based
                'Tempo_Visitas_Min': round(tempo_visitas, 1),
                'Tempo_Deslocamento_Min': round(tempo_deslocamento, 1),
                'Tempo_Total_Min': round(tempo_total, 1),
                'Jornada_Min': jornada,
                'Horas_Extras_Min': round(horas_extras, 1),
                'Limite_Total_Min': round(jornada + horas_extras, 1),
                'Folga_Min': round(jornada + horas_extras - tempo_total, 1),
                'Utilizacao_%': round(tempo_total / jornada * 100, 1) if jornada > 0 else 0,
                'Distancia_Rota_Total': round(distancia_total_rota, 2),
                'Status': 'OK' if tempo_total <= jornada + horas_extras + 0.01 else 'VIOLAÇÃO'
            }

            # Calcular deslocamento para cada loja baseado na rota otimizada
            deslocamento_por_loja = {loja: 0.0 for loja in todas_lojas}

            if len(rota) >= 2:
                for i in range(len(rota) - 1):
                    loja_origem = rota[i]
                    loja_destino = rota[i + 1]
                    dist = calcular_distancia(loja_origem, loja_destino)
                    deslocamento_por_loja[loja_destino] = dist

            # Adicionar coluna para cada loja
            for loja_id in todas_lojas:
                linha[f'Dist_Loja_{loja_id + 1}'] = round(deslocamento_por_loja[loja_id], 2)

            # Adicionar coluna de soma para verificação
            soma_dist_lojas = sum(deslocamento_por_loja.values())
            linha['Soma_Dist_Lojas'] = round(soma_dist_lojas, 2)
            linha['Diferenca_Verificacao'] = round(distancia_total_rota - soma_dist_lojas, 2)

            detalhamento_diario.append(linha)

    df_detalhamento = pd.DataFrame(detalhamento_diario)

    # Reordenar colunas para melhor visualização
    colunas_fixas = [
        'Promotor', 'Dia_Num', 'Dia_Nome', 'Num_Visitas', 'Lojas_Visitadas',
        'Rota_Otimizada', 'Tempo_Visitas_Min', 'Tempo_Deslocamento_Min',
        'Tempo_Total_Min', 'Jornada_Min', 'Horas_Extras_Min', 'Limite_Total_Min',
        'Folga_Min', 'Utilizacao_%', 'Distancia_Rota_Total'
    ]
    colunas_lojas = [f'Dist_Loja_{loja + 1}' for loja in todas_lojas]  # +1 para 1-based
    colunas_verificacao = ['Soma_Dist_Lojas', 'Diferenca_Verificacao', 'Status']

    # Garantir que todas as colunas existem
    colunas_ordenadas = [c for c in colunas_fixas + colunas_lojas + colunas_verificacao
                         if c in df_detalhamento.columns]
    df_detalhamento = df_detalhamento[colunas_ordenadas]

    df_detalhamento.to_excel(writer, sheet_name='Detalhamento_Diario', index=False)



    print(f"   Total de verificações: {total_verificacoes}")
    print(f"   OK: {total_ok} ({total_ok/total_verificacoes*100:.1f}%)")
    print(f"   Violações: {total_violacoes} ({total_violacoes/total_verificacoes*100:.1f}%)")

    if erros_encontrados == 0:
        print("Todas as restrições foram respeitadas")
    else:
        print(f"\n {erros_encontrados} violações encontradas")


print(f"\nArquivo Excel salvo em: {arquivo_excel}")

   Total de verificações: 145
   OK: 144 (99.3%)
   Violações: 1 (0.7%)

 1 violações encontradas

Arquivo Excel salvo em: resultado_otimizacao_heuristica.xlsx
